In [ ]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipywidgets as ipw

In [ ]:
def get_exp_data(test_num=1):
    skiprows = 3
    delimiter=';'
    decimal=','
    file_path = './results/T' + str(test_num) + '.csv'
    headers_array = list(np.array(pd.read_csv(file_path, delimiter=delimiter, decimal=decimal, nrows=1, header=None))[0])
    
    time_idx = 0
    force_idx = headers_array.index('Kraft')
    weg_idx = headers_array.index('Weg')
    IWA_M_hinten_idx = headers_array.index('IWA_M_hinten')
    IWA_M_vorne_idx = headers_array.index('IWA_M_vorne')
    DMS_idx = headers_array.index('DMS-1')
    
    data = np.array(pd.read_csv(file_path, delimiter=delimiter, decimal=decimal, skiprows=skiprows))
    time = data[:, 0]
    force = -data[:, force_idx]
    dms = -data[:, DMS_idx]
    weg = -data[:, weg_idx]
    hinten_vorne = - data[:, (IWA_M_hinten_idx, IWA_M_vorne_idx)]
    avg_hinten_vorne = (hinten_vorne[:, 0] + hinten_vorne[:, 1])/2
    
    return time, force, weg, avg_hinten_vorne, dms, hinten_vorne

def get_exp_fw(test_num=1):
    _, force, _, avg_hinten_vorne, _, hinten_vorne = get_exp_data(test_num = test_num)
    if test_num == 7:
        avg_hinten_vorne = hinten_vorne[:, 0]
    elif test_num == 8:
        avg_hinten_vorne = hinten_vorne[:, 1]
    return force, avg_hinten_vorne

In [ ]:
datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

In [ ]:
import sys

print(sys.version)

In [ ]:
'00:01:00.100'.split('.')[0],'%H:%M:%S'

In [ ]:
from datetime import date, datetime, time
from backports.datetime_fromisoformat import MonkeyPatch
MonkeyPatch.patch_fromisoformat()

In [ ]:
datetime.fromisoformat('07.03.2022 08:15:47.281')

In [ ]:
from datetime import date, datetime, time
import time
x = time.strptime('00:01:00.100','%H:%M:%S')
datetime.timedelta(hours=x.tm_hour, minutes=x.tm_min, seconds=x.tm_sec).total_seconds()


In [ ]:
path = r"C:\Users\hspartali\_Homam\Work\Projects\DFG-TRR 280\Experiments\01. M-k validation\Biegeversuche\FOS\T11_4PB_CS_0_5_2022-03-14_09-58-31_ch4_full.tsv"

skro = 31

with open(path, 'r+') as f:
    data_alt = pd.read_table(f, skiprows=skro, header=None, encoding='cp437', on_bad_lines='skip',
                             encoding_errors='ignore')
    data_alt = data_alt.drop([1, 2], axis=1)
    data_alt = data_alt.set_index(data_alt.columns[0])
    print(data_alt)
    tare = None
    if "Tare" in data_alt.index:
        tare = data_alt.loc["Tare"]
        data_alt = data_alt.drop(index="Tare")
    data_alt.set_axis(data_alt.loc["x-axis"], axis=1, inplace=True)
    data_alt = data_alt.drop(index="x-axis")
    data_alt.index.name = None
    data_alt.columns.name = None
    data_alt.index = pd.to_datetime(data_alt.index)
    data_alt = data_alt.astype(np.float64)
    data_alt.columns = np.float64(data_alt.columns)
    new_ind = pd.to_timedelta([(data_alt.index[i] - data_alt.index[0]) for i in range(len(data_alt.index))])
    data_alt = data_alt.set_index(new_ind)

# data_alt = data_alt.truncate(before=avoidLength, axis=1)

data_alt = data_alt.mul(1e-6)

In [ ]:
path = r"C:\Users\hspartali\_Homam\Work\Projects\DFG-TRR 280\Experiments\01. M-k validation\Biegeversuche\FOS\Raw data\T9_2022-03-07_07-15-13_ch4_full.tsv"

# Get number of rows to skip
with open(path) as file:
    i = 0
    skiprows = 0
    for line in file:
        i = i + 1
        if line == '----------------------------------------\n':
            skiprows = i
            break
    if skiprows == 0:
        print('Failed to find number of rows to skip (no line with dashes has been found)')

In [ ]:
df = pd.read_table(path, header=None, skiprows=skiprows)
df = df.drop([1, 2], axis=1)
# Drop time?
# df = df.drop([0], axis=1)

In [ ]:
df = df.astype(np.float64)

fill_nan = "LinInt Länge"
if fill_nan == "Spline-Interpolation":
    df = df.interpolate(method='spline', order=3, axis=1)
elif fill_nan == "LinInt Länge":
    df = df.interpolate(method='linear', axis=1)
elif fill_nan == "LinInt Zeit":
    df = df.interpolate(method='linear', axis=0)
elif fill_nan == "Ersetze mit 0":
    df = df.fillna(0)
elif fill_nan == "Lösche Spalten":
    df = df.dropna(axis=1)
elif fill_nan == "None":
    df = df
    
df_arr = np.array(df).astype(np.float_)

In [ ]:
# Calc y_limit
df_arr_except_1_line = df_arr[1:, :]
no_nan_df_arr = df_arr_except_1_line[~np.isnan(df_arr_except_1_line)]
y_limit = np.max(no_nan_df_arr)
y_limit

In [ ]:
test_num = 5

slider = ipw.SelectionSlider(
            options=np.arange(len(df_arr[:, 0])),
            value=0,
            orientation='horizontal',
            continuous_update=True,
#             tooltip=self.tooltip,
#             description=self.label,
#             disabled=self.disabled,
#             readout=self.readout,
#             style=style
        )


fig, axes = plt.subplots(1, 2)

output2 = ipw.Output()
display(slider, output2)
def on_value_change(change):
    with ipw.Output():
        new_value = change['new']
        axes[0].clear()
        axes[1].clear()
        axes[0].plot(df_arr[0, :], df_arr[new_value, :], '-')
        axes[0].set_ylim([-0.05 * y_limit, y_limit])
        fw = get_exp_fw(test_num = test_num)
        axes[1].plot(fw[1], fw[0])
        fig.show()
slider.observe(on_value_change, names='value')

## Plot 3D

In [ ]:
x = np.arange(df_arr.shape[0]) + np.zeros((df_arr.shape[0], 1))
y = df_arr[0, :] + np.zeros((df_arr.shape[0], 1))
z = df_arr

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)